In [1]:
%pylab inline
import numpy as np
import random
from keras.datasets import mnist

Populating the interactive namespace from numpy and matplotlib


In [2]:
# load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# normalize data between 0 and 1
# x_train = (x_train/255).astype(np.float64)
# x_test = (x_test/255).astype(np.float64)

In [3]:
def init_layer(m,n):
  # random weights between 1 and -1
  return np.random.uniform(-.1, .1, (m, n)).astype(np.float64)

In [4]:
l1 = init_layer(128, 28*28)
l2 = init_layer(10, 128)

In [5]:
# relu activation 
def relu(x):
  return np.maximum(x, 0)

# softmax activation
def softmax(x):
  # num = np.exp(x)
  # sum = np.exp(x).sum()
  # print("num", num, "softmax sum", sum)
  # return num/sum
  return np.exp(x - np.max(x)) / np.exp(x - np.max(x)).sum()

# forward and backward pass (SGD)
def single_pass(x, y):
  # convert y to one-hot vector
  y_oh = np.zeros((10, 1), dtype=np.float32)
  y_oh[y] = 1

  # flatten x and feed forward
  x = x.reshape((28*28, 1))
  x_l1 = l1.dot(x)
  x_relu = relu(x_l1)
  x_l2 = l2.dot(x_relu)
  x_sm = softmax(x_l2)
  y_pred = np.argmax(x_sm)

  x_ce = np.sum(-y_oh * np.log(x_sm))

  # d ce/l2_a | cross entropy
  # d_ce = -y_oh / x_sm
  
  # d ce/l2_a * l2_a/l2_z | softmax 
  d_sm = x_sm - y_oh

  # d ce/l2_a * l2_a/l2_z * l2_z/l2_w | l2 raw output
  d_l2_w = np.matmul(x_relu, d_sm.T)

  # d ce/l2_a * l2_a/l2_z * l2_z/l2_a
  d_l2 = l2.T.dot(d_sm)

  # d ce/l2_a * l2_a/l2_z * l2_z/l2_a * l2_a/l1_z | relu
  d_relu = d_l2 * (x_l1 > 0).astype(np.float32)

  # d ce/l2_a * l2_a/l2_z * l2_z/l2_a * l2_a/l1_z * l1_z/l1_w | l1 raw output
  d_l1_w = np.matmul(x, d_relu.T)

  return y_pred, d_l1_w, d_l2_w


In [6]:
def train(learning_rate, iterations):
  global l1, l2
  train = np.random.randint(0, x_train.shape[0], size=iterations)

  for i in range(iterations):
    x = x_train[train[i]]
    y = y_train[train[i]]
    y_pred, d_l1, d_l2 = single_pass(x, y)
    if i >= iterations - 10:
      print("iteration ", i + 1, " predicted: ", y_pred, "actual:", y)
    
    l1 = l1 - d_l1.T * learning_rate
    l2 = l2 - d_l2.T * learning_rate

In [7]:
def test(test_size):
  test = np.random.randint(0, x_test.shape[0], size=test_size)
  correct, total = 0,0
  for i in range(50):
    x = x_test[test[i]]
    y = y_test[test[i]]
    y_pred, d_l1, d_l2 = single_pass(x, y)
    total += 1
    if y_pred == y:
      correct += 1
    # print("iteration ", i + 1, " predicted: ", y_pred, "actual:", y)

  print("accuracy: ", correct/total)

In [8]:
train(0.00005, 5000)
test(y_train.size)

iteration  4991  predicted:  4 actual: 4
iteration  4992  predicted:  5 actual: 5
iteration  4993  predicted:  1 actual: 1
iteration  4994  predicted:  7 actual: 7
iteration  4995  predicted:  8 actual: 8
iteration  4996  predicted:  6 actual: 6
iteration  4997  predicted:  0 actual: 0
iteration  4998  predicted:  6 actual: 6
iteration  4999  predicted:  3 actual: 3
iteration  5000  predicted:  9 actual: 7
accuracy:  0.88
